# Merging Dataframes


In [172]:
from __future__ import division
import psycopg2
import pandas as pd
import numpy as np
import matplotlib as mpl
import pylab as plt
%matplotlib inline

In [173]:
# Connecting to redshift
con = psycopg2.connect(
host ='gl-analytics-production-1.cfzpbyicru2p.us-east-1.redshift.amazonaws.com',
port ='5439',
database = 'analysttest',
user = 'analysttest',
password ='yi3xigh4OoGho4uceire')

In [174]:
# Writing SQL query for daily first connection
DailyFC = pd.read_sql_query(
"""
SELECT 
    DISTINCT(account_id) as users,
    DATE_TRUNC('day',TIMESTAMP 'epoch' + "time" * INTERVAL '1 second ') as timestamp_main
FROM 
    tmnt.daily_first_connection
WHERE 
    DATE_TRUNC('day',TIMESTAMP 'epoch' + "time" * INTERVAL '1 second ') BETWEEN '2017-03-01 12:00:01' AND '2017-03-02 23:59:59'
""", con)

In [175]:
# Writing SQL query for paying users
Paying_user = pd.read_sql_query(
"""
SELECT 
    DISTINCT(account_id) as paying_user,
    product, 
    price,
    DATE_TRUNC('day',TIMESTAMP 'epoch' + "time" * INTERVAL '1 second ') as transaction_time
FROM 
    tmnt.verified_transactions
WHERE
    DATE_TRUNC('day',TIMESTAMP 'epoch' + "time" * INTERVAL '1 second ') BETWEEN '2017-03-01 12:00:01' AND '2017-03-02 23:59:59'
""", con)

In [176]:
DailyFC.head()

,users,timestamp_main
0,2600000358498,2017-03-02
1,2600020855817,2017-03-02
2,2600020868917,2017-03-02
3,2600020872937,2017-03-02
4,2600020356087,2017-03-02


In [177]:
len(DailyFC)

119149

In [178]:
Paying_user.head()

,paying_user,product,price,transaction_time
0,2600009317161,com.ludia.tmnt.cardpack.silver,19.99,2017-03-02
1,2600018918727,com.ludia.tmnt.subscription,9.99,2017-03-02
2,2600013432884,com.ludia.tmnt.warppass.3,9.99,2017-03-02
3,2600000627244,com.ludia.tmnt.hardcash.2,4.99,2017-03-02
4,2600016932586,com.ludia.tmnt.cardpack.mutagen.2,9.99,2017-03-02


In [179]:
len(Paying_user)

873

In [180]:
# Closing connection with redshift.
con.close()

In [181]:
# Checking the list of column names
list(Paying_user.columns)

['paying_user', 'product', 'price', 'transaction_time']

In [182]:
# Setting 'pyaing_user' column to index
Paying_user = Paying_user.set_index('paying_user')
Paying_user.head()

,product,price,transaction_time
paying_user,,,
2600009317161,com.ludia.tmnt.cardpack.silver,19.99,2017-03-02
2600018918727,com.ludia.tmnt.subscription,9.99,2017-03-02
2600013432884,com.ludia.tmnt.warppass.3,9.99,2017-03-02
2600000627244,com.ludia.tmnt.hardcash.2,4.99,2017-03-02
2600016932586,com.ludia.tmnt.cardpack.mutagen.2,9.99,2017-03-02


In [183]:
len(Paying_user)

873

In [184]:
# Setting 'users' to index
DailyFC = DailyFC.set_index('users')
DailyFC.head()

,timestamp_main
users,
2600000358498,2017-03-02
2600020855817,2017-03-02
2600020868917,2017-03-02
2600020872937,2017-03-02
2600020356087,2017-03-02


In [185]:
len(DailyFC)

119149

In [186]:
# Merging datasets with each other. 
pd.merge(DailyFC, Paying_user, how='outer', left_index=True, right_index=True).head()

,timestamp_main,product,price,transaction_time
0,2017-03-02,NaN,NaN,NaT
2600000000005,2017-03-02,NaN,NaN,NaT
2600000000046,2017-03-02,NaN,NaN,NaT
2600000000080,2017-03-02,NaN,NaN,NaT
2600000000221,2017-03-02,NaN,NaN,NaT


In [187]:
pd.merge(DailyFC, Paying_user, how='inner', left_index=True, right_index=True).head()

,timestamp_main,product,price,transaction_time
2600000000796,2017-03-02,com.ludia.tmnt.warppass.1,1.99,2017-03-02
2600000000796,2017-03-02,com.ludia.tmnt.hardcash.2,4.99,2017-03-02
2600000000796,2017-03-02,com.ludia.tmnt.cardpack.gold.50off,24.99,2017-03-02
2600000000796,2017-03-02,com.ludia.tmnt.hardcash.1,1.99,2017-03-02
2600000000796,2017-03-02,com.ludia.tmnt.hardcash.2b,9.99,2017-03-02


In [188]:
pd.merge(DailyFC, Paying_user, how='left', left_index=True, right_index=True).head()

,timestamp_main,product,price,transaction_time
0,2017-03-02,NaN,NaN,NaT
2600000000005,2017-03-02,NaN,NaN,NaT
2600000000046,2017-03-02,NaN,NaN,NaT
2600000000080,2017-03-02,NaN,NaN,NaT
2600000000221,2017-03-02,NaN,NaN,NaT


In [189]:
pd.merge(DailyFC, Paying_user, how='right', left_index=True, right_index=True).head()

,timestamp_main,product,price,transaction_time
2600000000796,2017-03-02,com.ludia.tmnt.warppass.1,1.99,2017-03-02
2600000000796,2017-03-02,com.ludia.tmnt.hardcash.2,4.99,2017-03-02
2600000000796,2017-03-02,com.ludia.tmnt.cardpack.gold.50off,24.99,2017-03-02
2600000000796,2017-03-02,com.ludia.tmnt.hardcash.1,1.99,2017-03-02
2600000000796,2017-03-02,com.ludia.tmnt.hardcash.2b,9.99,2017-03-02


In [190]:
# It's important to reset the index 
DailyFC = DailyFC.reset_index()
Paying_user = Paying_user.reset_index()
pd.merge(DailyFC, Paying_user, how='left', left_on='users', right_on='paying_user').head()

,users,timestamp_main,paying_user,product,price,transaction_time
0,2600000358498,2017-03-02,NaN,NaN,NaN,NaT
1,2600020855817,2017-03-02,NaN,NaN,NaN,NaT
2,2600020868917,2017-03-02,NaN,NaN,NaN,NaT
3,2600020872937,2017-03-02,NaN,NaN,NaN,NaT
4,2600020356087,2017-03-02,NaN,NaN,NaN,NaT


# Idiomatic Pandas: Making Code Pandorable

In [191]:
import pandas as pd
Paying_user.head()

,paying_user,product,price,transaction_time
0,2600009317161,com.ludia.tmnt.cardpack.silver,19.99,2017-03-02
1,2600018918727,com.ludia.tmnt.subscription,9.99,2017-03-02
2,2600013432884,com.ludia.tmnt.warppass.3,9.99,2017-03-02
3,2600000627244,com.ludia.tmnt.hardcash.2,4.99,2017-03-02
4,2600016932586,com.ludia.tmnt.cardpack.mutagen.2,9.99,2017-03-02


In [192]:
# Taking all the price with 9.99.
# Removing NA if any
# Setting index
# Changing the column name

(Paying_user.where(Paying_user['price']==9.99)
    .dropna()
    .set_index(['paying_user','product'])
    .rename(columns={'price': 'amount'})).head()

,,amount,transaction_time
paying_user,product,,
2600018918727,com.ludia.tmnt.subscription,9.99,2017-03-02
2600013432884,com.ludia.tmnt.warppass.3,9.99,2017-03-02
2600016932586,com.ludia.tmnt.cardpack.mutagen.2,9.99,2017-03-02
2600001527506,com.ludia.tmnt.warppass.3,9.99,2017-03-02
2600005489469,com.ludia.tmnt.hardcash.2b,9.99,2017-03-02


In [193]:
# Taking all the price with 9.99.
# Removing NA if any
# Setting index
# Changing the column name
# Dropping transaction_Time column

(Paying_user.where(Paying_user['price']==9.99)
    .dropna()
    .drop('transaction_time', axis=1)
    .set_index(['paying_user','product'])
    .rename(columns={'price': 'amount'})).head()

,,amount
paying_user,product,
2600018918727,com.ludia.tmnt.subscription,9.99
2600013432884,com.ludia.tmnt.warppass.3,9.99
2600016932586,com.ludia.tmnt.cardpack.mutagen.2,9.99
2600001527506,com.ludia.tmnt.warppass.3,9.99
2600005489469,com.ludia.tmnt.hardcash.2b,9.99


In [194]:
# How to write it not being pandorable
Paying_user = Paying_user[Paying_user['price']==9.99]
Paying_user.set_index(['paying_user','product'], inplace=True)
Paying_user.rename(columns={'price': 'amount'}).head()

,,amount,transaction_time
paying_user,product,,
2600018918727,com.ludia.tmnt.subscription,9.99,2017-03-02
2600013432884,com.ludia.tmnt.warppass.3,9.99,2017-03-02
2600016932586,com.ludia.tmnt.cardpack.mutagen.2,9.99,2017-03-02
2600001527506,com.ludia.tmnt.warppass.3,9.99,2017-03-02
2600005489469,com.ludia.tmnt.hardcash.2b,9.99,2017-03-02


In [195]:
# New dataset (For practicing)
import pandas as pd
df = pd.read_csv('census.csv')
df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243286,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [24]:
len(df)

3193

In [214]:
#list(df.columns)

In [196]:
# Select SUMLEV==50
# Drop all NA
# Set_index to STANME and CTYNAME
# Rename column ESTIMATESBASE2010 to Estimates Base 2010
(df.where(df['SUMLEV']==50)
    .dropna()
    .set_index(['STNAME', 'CTYNAME'])
    .rename(columns={'ESTIMATESBASE2010': 'Estimates Base 2010'})).head()


SUMLEV  REGION  DIVISION  STATE  COUNTY  \
STNAME  CTYNAME                                                   
Alabama Autauga County    50.0     3.0       6.0    1.0     1.0   
        Baldwin County    50.0     3.0       6.0    1.0     3.0   
        Barbour County    50.0     3.0       6.0    1.0     5.0   
        Bibb County       50.0     3.0       6.0    1.0     7.0   
        Blount County     50.0     3.0       6.0    1.0     9.0   

                        CENSUS2010POP  Estimates Base 2010  POPESTIMATE2010  \
STNAME  CTYNAME                                                               
Alabama Autauga County        54571.0              54571.0          54660.0   
        Baldwin County       182265.0             182265.0         183193.0   
        Barbour County        27457.0              27457.0          27341.0   
        Bibb County           22915.0              22919.0          22861.0   
        Blount County         57322.0              57322.0          57373.0   

                        POPESTIMATE2011  POPESTIMATE2012     ...       \
STNAME  CTYNAME                                              ...        
Alabama Autauga County          55253.0          55175.0     ...        
        Baldwin County         186659.0         190396.0     ...        
        Barbour County          27226.0          27159.0     ...        
        Bibb County             22733.0          22642.0     ...        
        Blount County           57711.0          57776.0     ...        

                        RDOMESTICMIG2011  RDOMESTICMIG2012  RDOMESTICMIG2013  \
STNAME  CTYNAME                                                                
Alabama Autauga County          7.242091         -2.915927         -3.012349   
        Baldwin County         14.832960         17.647293         21.845705   
        Barbour County         -4.728132         -2.500690         -7.056824   
        Bibb County            -5.527043         -5.068871         -6.201001   
        Blount County           1.807375         -1.177622         -1.748766   

                        RDOMESTICMIG2014  RDOMESTICMIG2015  RNETMIG2011  \
STNAME  CTYNAME                                                           
Alabama Autauga County          2.265971         -2.530799     7.606016   
        Baldwin County         19.243286         17.197872    15.844176   
        Barbour County         -3.904217        -10.543299    -4.874741   
        Bibb County            -0.177537          0.177258    -5.088389   
        Blount County          -2.062535         -1.369970     1.859511   

                        RNETMIG2012  RNETMIG2013  RNETMIG2014  RNETMIG2015  
STNAME  CTYNAME                                                             
Alabama Autauga County    -2.626146    -2.722002     2.592270    -2.187333  
        Baldwin County    18.559627    22.727626    20.317142    18.293499  
        Barbour County    -2.758113    -7.167664    -3.978583   -10.543299  
        Bibb County       -4.363636    -5.403729     0.754533     1.107861  
        Blount County     -0.848580    -1.402476    -1.577232    -0.884411  

[5 rows x 98 columns]

In [156]:
# How to calcuate maximum and minimum of each POPESTIMATE from 2010 to 2015 in each row?
import numpy as np

In [197]:
def max_min(col):
    data = col[['POPESTIMATE2010',
                'POPESTIMATE2011',
                'POPESTIMATE2012',
                'POPESTIMATE2013',
                'POPESTIMATE2014',
                'POPESTIMATE2015']]
    return pd.Series({'max':np.max(data), 'min':np.min(data)})
    

In [198]:
df.apply(max_min, axis=1).head()

,max,min
0,4858979,4785161
1,55347,54660
2,203709,183193
3,27341,26489
4,22861,22512


In [199]:
# How to calculate min and max of each row and add them as a column
import numpy as np
def min_max(col):
    data = col[['POPESTIMATE2010',
                'POPESTIMATE2011',
                'POPESTIMATE2012',
                'POPESTIMATE2013',
                'POPESTIMATE2014',
                'POPESTIMATE2015']]
    col['max'] = np.max(data)
    col['min'] = np.min(data)
    return col
df.apply(min_max, axis=1).head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,max,min
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594,4858979,4785161
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333,55347,54660
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,21.845705,19.243286,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499,203709,183193
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299,27341,26489
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861,22861,22512


In [200]:
col = ['POPESTIMATE2010',
        'POPESTIMATE2011',
        'POPESTIMATE2012',
        'POPESTIMATE2013',
        'POPESTIMATE2014',
        'POPESTIMATE2015']
df.apply(lambda x: np.max(x[col]), axis=1).head()

0    4858979
1      55347
2     203709
3      27341
4      22861
dtype: int64

# Group by

In [218]:
import pandas as pd
import numpy as np
df = pd.read_csv('census.csv')
df = df[df['SUMLEV']==50]
df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243286,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861
5,50,3,6,1,9,Alabama,Blount County,57322,57322,57373,...,1.807375,-1.177622,-1.748766,-2.062535,-1.369970,1.859511,-0.848580,-1.402476,-1.577232,-0.884411


In [ ]:
%%timeit -n 10
# How to calcualte average of population for each STATE based on the number of counties. 
for state in df['STNAME'].unique():
    avg = np.average(df.where(df['STNAME']==state).dropna()['CENSUS2010POP'])
    print('Counties in state ' + state + ' have an average population of ' + str(avg))

In [ ]:
%%timeit -n 10
# More efficeint way
for group, frame in df.groupby('STNAME'):
    avg = np.average(frame['CENSUS2010POP'])
    print('Counties in state ' + group + ' have an average population of ' + str(avg))

In [204]:
# Reading census data again.
df = pd.read_csv('census.csv')
df = df[df['SUMLEV']==50]

In [205]:
# How to use groupby?
df.groupby('STNAME').agg({'CENSUS2010POP': np.average}).head()

,CENSUS2010POP
STNAME,
Alabama,71339.343284
Alaska,24490.724138
Arizona,426134.466667
Arkansas,38878.906667
California,642309.586207


In [71]:
print(type(df.groupby(level=0)['POPESTIMATE2010','POPESTIMATE2011']))
print(type(df.groupby(level=0)['POPESTIMATE2010']))

<class 'pandas.core.groupby.DataFrameGroupBy'>
<class 'pandas.core.groupby.SeriesGroupBy'>


In [207]:
# Calculating avg and sum of each STANAME for CENSUS2010POP (Pandorable way)
(df.set_index('STNAME').groupby(level=0)['CENSUS2010POP']
    .agg({'avg': np.average, 'sum': np.sum})).head()

,avg,sum
STNAME,,
Alabama,71339.343284,4779736
Alaska,24490.724138,710231
Arizona,426134.466667,6392017
Arkansas,38878.906667,2915918
California,642309.586207,37253956


In [208]:
(df.set_index('STNAME').groupby(level=0)['POPESTIMATE2010','POPESTIMATE2011']
    .agg({'avg': np.average, 'sum': np.sum})).head()

avg                             sum                
           POPESTIMATE2010 POPESTIMATE2011 POPESTIMATE2010 POPESTIMATE2011
STNAME                                                                    
Alabama       71420.313433    71658.328358         4785161         4801108
Alaska        24621.413793    24921.379310          714021          722720
Arizona      427213.866667   431248.800000         6408208         6468732
Arkansas      38965.253333    39180.506667         2922394         2938538
California   643691.017241   650000.586207        37334079        37700034

In [209]:
# Renaming the col name(Pandorable way)
(df.set_index('STNAME').groupby(level=0)['POPESTIMATE2010','POPESTIMATE2011']
    .agg({'POPESTIMATE2010': np.average, 'POPESTIMATE2011': np.sum})).head()

,POPESTIMATE2010,POPESTIMATE2011
STNAME,,
Alabama,71420.313433,4801108
Alaska,24621.413793,722720
Arizona,427213.866667,6468732
Arkansas,38965.253333,2938538
California,643691.017241,37700034


# Pivot Tables

In [210]:
#http://open.canada.ca/data/en/dataset/98f1a129-f628-4ce4-b24d-6f16bf24dd64
df = pd.read_csv('cars.csv')

In [211]:
df.head()

,YEAR,Make,Model,Size,(kW),Unnamed: 5,TYPE,CITY (kWh/100 km),HWY (kWh/100 km),COMB (kWh/100 km),CITY (Le/100 km),HWY (Le/100 km),COMB (Le/100 km),(g/km),RATING,(km),TIME (h)
0,2012,MITSUBISHI,i-MiEV,SUBCOMPACT,49,A1,B,16.9,21.4,18.7,1.9,2.4,2.1,0,n/a,100,7
1,2012,NISSAN,LEAF,MID-SIZE,80,A1,B,19.3,23.0,21.1,2.2,2.6,2.4,0,n/a,117,7
2,2013,FORD,FOCUS ELECTRIC,COMPACT,107,A1,B,19.0,21.1,20.0,2.1,2.4,2.2,0,n/a,122,4
3,2013,MITSUBISHI,i-MiEV,SUBCOMPACT,49,A1,B,16.9,21.4,18.7,1.9,2.4,2.1,0,n/a,100,7
4,2013,NISSAN,LEAF,MID-SIZE,80,A1,B,19.3,23.0,21.1,2.2,2.6,2.4,0,n/a,117,7


In [212]:
# Changing Make column values to the dataset columns
df.pivot_table(values='(kW)', index='YEAR', columns='Make', aggfunc=np.mean).head()

Make,BMW,CHEVROLET,FORD,KIA,MITSUBISHI,NISSAN,SMART,TESLA
YEAR,,,,,,,,
2012,NaN,NaN,NaN,NaN,49.0,80.0,NaN,NaN
2013,NaN,NaN,107.0,NaN,49.0,80.0,35.0,280.000000
2014,NaN,104.0,107.0,NaN,49.0,80.0,35.0,268.333333
2015,125.0,104.0,107.0,81.0,49.0,80.0,35.0,320.666667
2016,125.0,104.0,107.0,81.0,49.0,80.0,35.0,409.700000


In [213]:
df.pivot_table(values='(kW)', index='YEAR', columns='Make', aggfunc=[np.mean, np.min], margins=True).head()

mean                                                             \
Make    BMW CHEVROLET   FORD   KIA MITSUBISHI NISSAN SMART       TESLA   
YEAR                                                                     
2012    NaN       NaN    NaN   NaN       49.0   80.0   NaN         NaN   
2013    NaN       NaN  107.0   NaN       49.0   80.0  35.0  280.000000   
2014    NaN     104.0  107.0   NaN       49.0   80.0  35.0  268.333333   
2015  125.0     104.0  107.0  81.0       49.0   80.0  35.0  320.666667   
2016  125.0     104.0  107.0  81.0       49.0   80.0  35.0  409.700000   

                   amin                                                        \
Make         All    BMW CHEVROLET   FORD   KIA MITSUBISHI NISSAN SMART  TESLA   
YEAR                                                                            
2012   64.500000    NaN       NaN    NaN   NaN       49.0   80.0   NaN    NaN   
2013  158.444444    NaN       NaN  107.0   NaN       49.0   80.0  35.0  270.0   
2014  135.000000    NaN     104.0  107.0   NaN       49.0   80.0  35.0  225.0   
2015  181.428571  125.0     104.0  107.0  81.0       49.0   80.0  35.0  280.0   
2016  252.263158  125.0     104.0  107.0  81.0       49.0   80.0  35.0  283.0   

            
Make   All  
YEAR        
2012  49.0  
2013  35.0  
2014  35.0  
2015  35.0  
2016  35.0